In [6]:
from __future__ import unicode_literals
import os, os.path
import fnmatch
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
import nltk
import json
#nltk.download('averaged_perceptron_tagger')
#nltk.download('maxent_ne_chunker')

import pycountry
import csv
import sys
# reload(sys)
# sys.setdefaultencoding('utf-8')

In [7]:
#nltk.download('words')
# nltk.download('all')
with open('config.json') as handle:
    config = json.load(handle)
book = config['book_name']
Parts = config['number_of_parts']
dir_path = './%s/part'% book

In [8]:
chapter_entities_map = {}
entity_list_map = {}
notation_int_map = {}
location_list_map = {}
# 0 = PERSON 1 = PLACE
notation_int_map['PERSON'] = 0
notation_int_map['GPE'] = 1

In [9]:
# for finding location list

countries = {}
city = {}
for c in pycountry.countries:
    countries[c.name] = 1
with open ('city.csv','r', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    for raw in reader:
        city[raw['city_name']] = 1

In [10]:
for part in range(1,Parts+1):
    
    chapter_entities_map[part] = {}
    entity_list_map[part] = {}
    location_list_map[part] = {}
    x = 1
    print(dir_path + str(part) + '/')
    no_of_chapters = len(fnmatch.filter(os.listdir(dir_path + str(part) + '/'), '*.txt'))
    print('no.of_chapters:' + str(no_of_chapters))
    print('part:' + str(part))
    while x <= no_of_chapters:
        
        target_x = no_of_chapters+1
        print('chapters' + str(x) + ' to ' + str(target_x) )
        for i in range(x,target_x):
            with open(dir_path + str(part) + '/chapter'+ str(i) + '.txt', 'r', encoding='utf-8') as content_file:
                content = content_file.read()
            chapter_entities_map[part][i] = set()
            location_list_map[part][i] = set()
#             content = unicode(content, "utf-8")
            
            for c in city:
                if c in content:
                    location_list_map[part][i].add(c)
            for c in countries:
                if c in content:
                    location_list_map[part][i].add(c)
                    
                    
            for sent in nltk.sent_tokenize(content):
                for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent))):
                    if not hasattr(chunk, 'label'):
                        continue
                    if chunk.label() not in notation_int_map.keys():
                        continue

                    entity = ' '.join(c[0] for c in chunk)
                    # print(chunk.label(), entity)

                    chapter_entities_map[part][i].add(entity)

                    if entity not in entity_list_map.keys():
                        entity_list_map[part][entity] = []  
                    entity_list_map[part][entity].append(notation_int_map[chunk.label()])
            chapter_entities_map[part][i] = list(chapter_entities_map[part][i])
            location_list_map[part][i] = list(location_list_map[part][i])
        #print chapter_entities_map
        
        x = target_x
        
with open('json_files/%s/nltk_chapter_entities_map.json' % book, 'w') as handle:
    json.dump(chapter_entities_map,handle,indent=4)
#print entity_list_map
with open('json_files/%s/nltk_entity_list_map.json'% book, 'w') as handle:
    json.dump(entity_list_map,handle,indent=4)
with open('json_files/%s/location_list_map.json'% book, 'w') as handle:
    json.dump(location_list_map,handle,indent=4)


./Lenin_Selected_Works/part1/
no.of_chapters:27
part:1
chapters1 to 28
